# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Metal device set to: Apple M1


2024-03-07 20:49:48.548708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-07 20:49:48.550228: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/150


2024-03-07 20:49:48.873057: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-07 20:49:49.036022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - ETA: 0s - loss: nan - mse: nan                        

2024-03-07 20:49:50.467042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 2s 27ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 6ms

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))# Train the model

Epoch 1/150
11/33 [=========>....................] - ETA: 0s - loss: nan - mse: nan                          

2024-03-07 20:50:46.197101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 10ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
22/33 [===================>..........] - ETA: 0s - loss: nan - mse: nan

2024-03-07 20:50:46.537230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 6ms

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)


In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Epoch 1/150
10/33 [========>.....................] - ETA: 0s - loss: 0.8458 - mse: 0.8458

2024-03-07 20:51:26.924491: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 10ms/step - loss: 0.5166 - mse: 0.5166 - val_loss: 0.1928 - val_mse: 0.1928
Epoch 2/150
22/33 [===================>..........] - ETA: 0s - loss: 0.2853 - mse: 0.2853

2024-03-07 20:51:27.260312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.2485 - mse: 0.2485 - val_loss: 0.1499 - val_mse: 0.1499
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1920 - mse: 0.1920 - val_loss: 0.1413 - val_mse: 0.1413
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1658 - mse: 0.1658 - val_loss: 0.1358 - val_mse: 0.1358
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1429 - mse: 0.1429 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1252 - mse: 0.1252 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1150 - mse: 0.1150 - val_loss: 0.1248 - val_mse: 0.1248
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1065 - mse: 0.1065 - val_loss: 0.1264 - val_mse: 0.1264
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0996 - mse: 0.0996 -

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 4ms/step - loss: 0.0063 - mse: 0.0063


[0.006348082795739174, 0.006348082795739174]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 4ms/step - loss: 0.0995 - mse: 0.0995


[0.09951493889093399, 0.09951493889093399]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))


9/9 [==============================] - 0s 4ms/step


2024-03-07 20:52:20.206381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24789.94841373887

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))


Epoch 1/150
 9/33 [=======>......................] - ETA: 0s - loss: 0.7118 - mse: 0.7118

2024-03-07 20:52:39.406056: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 10ms/step - loss: 0.4501 - mse: 0.4501 - val_loss: 0.2398 - val_mse: 0.2398
Epoch 2/150
23/33 [===================>..........] - ETA: 0s - loss: 0.2891 - mse: 0.2891

2024-03-07 20:52:39.745504: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.2464 - mse: 0.2464 - val_loss: 0.1878 - val_mse: 0.1878
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1939 - mse: 0.1939 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1650 - mse: 0.1650 - val_loss: 0.1655 - val_mse: 0.1655
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1490 - mse: 0.1490 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1293 - mse: 0.1293 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1229 - mse: 0.1229 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1099 - mse: 0.1099 - val_loss: 0.1456 - val_mse: 0.1456
Epoch 9/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1034 - mse: 0.1034 -

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data


Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [ ]:
# Evaluate the model on validate data


## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [ ]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model


# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 4ms/step - loss: 0.0090 - mse: 0.0090


[0.00899476744234562, 0.00899476744234562]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 4ms/step - loss: 0.1338 - mse: 0.1338


[0.13379260897636414, 0.13379260897636414]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [18]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150


2024-03-07 20:54:07.123300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 13ms/step - loss: 0.4154 - mse: 0.4154 - val_loss: 0.1552 - val_mse: 0.1552
Epoch 2/150
 8/33 [======>.......................] - ETA: 0s - loss: 0.2471 - mse: 0.2471

2024-03-07 20:54:07.956597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: 0.1846 - mse: 0.1846 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 3/150
33/33 [==============================] - 0s 9ms/step - loss: 0.1454 - mse: 0.1454 - val_loss: 0.1278 - val_mse: 0.1278
Epoch 4/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0890 - mse: 0.0890 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 5/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0881 - mse: 0.0881 - val_loss: 0.1103 - val_mse: 0.1103
Epoch 6/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0594 - mse: 0.0594 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 7/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0584 - mse: 0.0584 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 8/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0501 - mse: 0.0501 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 9/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0372 - mse: 0.0372 -

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)


33/33 [==============================] - 0s 4ms/step - loss: 0.0043 - mse: 0.0043


[0.004265077877789736, 0.004265077877789736]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [20]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)


9/9 [==============================] - 0s 4ms/step - loss: 0.1002 - mse: 0.1002


[0.10020484030246735, 0.10020484030246735]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [ ]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150


2024-03-07 20:54:53.760655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 12ms/step - loss: 0.5535 - mse: 0.5535 - val_loss: 0.1793 - val_mse: 0.1793
Epoch 2/150
20/33 [=================>............] - ETA: 0s - loss: 0.1133 - mse: 0.1133

2024-03-07 20:54:54.594180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 7ms/step - loss: 0.1853 - mse: 0.1853 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1282 - mse: 0.1282 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 4/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0906 - mse: 0.0906 - val_loss: 0.1043 - val_mse: 0.1043
Epoch 5/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0724 - mse: 0.0724 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 6/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0527 - mse: 0.0527 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.0476 - mse: 0.0476 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 8/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0361 - mse: 0.0361 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0301 - mse: 0.0301 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)


Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [ ]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [ ]:
# Evaluate the best model on test data


As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [ ]:
# Generate predictions on test data
y_test_pred_scaled = None

# Transform the predictions back to original scale
y_test_pred = None

# MSE of test data


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.